In [3]:
import sys
import torch
import numpy as np
import pickle as pkl
from tqdm import tqdm

sys.path.append('../../../')

from experiments.assumptions.degeneracy.script import eigenvalue_result, eigenvalue_results_large, plot_rank_train, rank_over_training
from models.supervised.mlp.model import MLP
from models.supervised.bimt.model import BioMLP



In [4]:
np.random.seed(2)
torch.manual_seed(2)

In [6]:
mode = "moon"
model_name = "mlp"
size = "2_wide"
models_path = f"../../../models/supervised/{model_name}/saved_models"
if size == "overfit":
	with open(f'{models_path}/2_wide/{mode}/dataset.pkl', 'rb') as f:
		dataset = pkl.load(f)
else:
	with open(f'{models_path}/{size}/{mode}/dataset.pkl', 'rb') as f:
		dataset = pkl.load(f)

for size in ["mixup_vanilla", "vanilla",  "overfit", "2_wide"]:
	if size == "overfit":
		epochs = [60, 80, 199, 9999, 999, 5]
	else:
		epochs = [5, 60, 80, 199]
	for epoch in epochs:
		if size == "vanilla" or size == "mixup_vanilla":
			model = MLP(2,7,10,2)
		elif size == "overfit":
			model = MLP(2,7,2,1)
		elif size == "2_wide":
			model = MLP(2,7,2,2)
		print(f"Size: {size}, Epoch: {epoch}")
		model.eval()
		model.load_state_dict(torch.load(f'{models_path}/{size}/{mode}/model_{epoch}.pth'))
		if size == "overfit":
			model.num_layers -= 1
			model.layers = model.layers[:-1]
		save_path = f"figures/{model_name}/{mode}/{size}/{epoch}/"
		eigenvalue_result(dataset.X, model, N=50, labels=dataset.y, wrt="output_wise", sigma=0.05, precision=7, save_path=save_path)
		eigenvalue_result(dataset.X, model, N=50, labels=dataset.y, wrt="layer_wise", sigma=0.05, precision=7, save_path=save_path)

Size: vanilla, Epoch: 5
Size: vanilla, Epoch: 60
Size: vanilla, Epoch: 80
Size: vanilla, Epoch: 199


In [ ]:
mode = "blobs"
model_name = "mlp"
size = "vanilla"
with open(f'{models_path}/{size}/{mode}/dataset.pkl', 'rb') as f:
	dataset = pkl.load(f)
epochs = [5, 60, 80, 199]

for size in ["vanilla"]:
	for epoch in epochs:
		if size == "vanilla":
			model = MLP(2,7,4,4)
		print(f"Size: {size}, Epoch: {epoch}")
		model.eval()
		model.load_state_dict(torch.load(f'{models_path}/{size}/{mode}/model_{epoch}.pth'))
		save_path = f"figures/{model_name}/{mode}/{size}/{epoch}/"
		eigenvalue_result(dataset.X, model, N=50, labels=dataset.y, wrt="output_wise", sigma=0.05, precision=7, save_path=save_path)
		eigenvalue_result(dataset.X, model, N=50, labels=dataset.y, wrt="layer_wise", sigma=0.05, precision=7, save_path=save_path)

In [4]:
mode="moon"
model_name = "bimt"
size = "vanilla"

model = BioMLP(shp=[2,20,20,2])
models_path = f"../../../models/supervised/{model_name}/saved_models"

res_q_25, res_med, res_q_75 = [], [], []
with open(f'{models_path}/{size}/{mode}/dataset.pkl', 'rb') as f:
	dataset = pkl.load(f)

for epoch in tqdm(range(0, 10000, 100)):
	model.load_state_dict(torch.load(f'{models_path}/{size}/{mode}/model_{epoch}.pth'))
	model.eval()
	save_path = f"figures/{model_name}/{mode}/{size}/{epoch}/"
	
	q_25, med, q_75 = eigenvalue_results_large(dataset.X, model, N=50, wrt="output_wise", sigma=0.05, precision=7, save_path=save_path)
	res_q_25.append(q_25)
	res_med.append(med)
	res_q_75.append(q_75)


	

100%|██████████| 100/100 [02:32<00:00,  1.52s/it]


In [ ]:
plot_rank_train(res_q_25, res_med, res_q_75, savepath=f"figures/{model_name}/{mode}/{size}/")


In [7]:
mode="moon"
model_name = "mlp"
size = "overfit"


models_path = f"../../../models/supervised/{model_name}/saved_models"

res_q_25, res_med, res_q_75 = [], [], []
with open(f'{models_path}/2_wide/{mode}/dataset.pkl', 'rb') as f:
	dataset = pkl.load(f)

for epoch in tqdm(range(0, 10000, 100)):
	model = MLP(2,7,2,1)

	model.load_state_dict(torch.load(f'{models_path}/{size}/{mode}/model_{epoch}.pth'))
	if size == "overfit":
		model.num_layers -= 1
		model.layers = model.layers[:-1]
	model.eval()
	save_path = f"figures/{model_name}/{mode}/{size}/{epoch}/"
	
	q_25, med, q_75 = rank_over_training(dataset.X, model, N=50, wrt="output_wise", sigma=0.05, precision=7)
	res_q_25.append(q_25)
	res_med.append(med)
	res_q_75.append(q_75)

100%|██████████| 100/100 [00:35<00:00,  2.81it/s]


In [8]:
plot_rank_train(res_q_25, res_med, res_q_75, savepath=f"figures/{model_name}/{mode}/{size}/")
